In [1]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snspy

Matplotlib is building the font cache; this may take a moment.


# PS Standard Test

In [2]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def extract_benchmark_data():
    """
    Extract benchmark data for both proving and verification times from PS protocol benchmarks
    """
    results = []
    base_path = Path("../target/criterion/ps_equality_split")

    print(f"Analyzing benchmark data from: {base_path}")

    # Look for both prove and verify benchmarks
    prove_dirs = list(base_path.glob("prove_credentials_*"))
    verify_dirs = list(base_path.glob("verify_credentials_*"))

    print(
        f"\nFound {len(prove_dirs)} proving and {len(verify_dirs)} verification configurations")

    # Process proving benchmarks
    for bench_dir in prove_dirs:
        # Extract credential count from directory name
        creds = int(bench_dir.name.split('_')[2])

        estimates_file = bench_dir / "new" / "estimates.json"
        print(f"\nAnalyzing proving configuration: {creds}")
        print(f"Reading from: {estimates_file}")

        if estimates_file.exists():
            try:
                with open(estimates_file, 'r') as f:
                    data = json.load(f)

                # Extract timing metrics (converting nanoseconds to milliseconds)
                mean_time = data['mean']['point_estimate'] / 1_000_000
                std_dev = data['mean']['standard_error'] / 1_000_000

                # Find corresponding verification time
                verify_dir = next(d for d in verify_dirs if int(
                    d.name.split('_')[2]) == creds)
                verify_estimates = verify_dir / "new" / "estimates.json"

                with open(verify_estimates, 'r') as f:
                    verify_data = json.load(f)

                verify_mean = verify_data['mean']['point_estimate'] / 1_000_000
                verify_std = verify_data['mean']['standard_error'] / 1_000_000

                results.append({
                    'Credentials': creds,
                    'Prove Time (ms)': mean_time,
                    'Prove Std Dev': std_dev,
                    'Verify Time (ms)': verify_mean,
                    'Verify Std Dev': verify_std,
                    'Total Time (ms)': mean_time + verify_mean
                })
                print(f"✓ Successfully processed data for {creds} credentials")

            except Exception as e:
                print(f"× Error processing data: {e}")

    if results:
        # Create DataFrame and sort by credential count
        df = pd.DataFrame(results)
        df = df.sort_values('Credentials')

        # Save results
        df.to_csv('ps_credential_scaling.csv', index=False)
        df.to_json('ps_credential_scaling.json', orient='records', indent=2)

        print("\nScaling Analysis Summary:")
        print("-" * 70)
        print(df.to_string(index=False))

        return df
    else:
        print("\nNo data was collected!")
        return None


def analyze_scaling(df):
    """Analyze and visualize scaling behavior for PS protocol proving and verification"""
    plt.figure(figsize=(12, 8))

    # Plot proving time with error bars
    plt.scatter(df['Credentials'], df['Prove Time (ms)'],
                color='blue', label='Proving Time', s=100)
    plt.errorbar(df['Credentials'], df['Prove Time (ms)'],
                 yerr=df['Prove Std Dev'], fmt='none',
                 capsize=5, color='blue', alpha=0.5)

    # Plot verification time with error bars
    plt.scatter(df['Credentials'], df['Verify Time (ms)'],
                color='red', label='Verification Time', s=100)
    plt.errorbar(df['Credentials'], df['Verify Time (ms)'],
                 yerr=df['Verify Std Dev'], fmt='none',
                 capsize=5, color='red', alpha=0.5)

    # Plot total time
    plt.scatter(df['Credentials'], df['Total Time (ms)'],
                color='green', label='Total Time', s=100)

    # Compute and plot ideal linear scaling for both operations
    base_creds = df['Credentials'].iloc[0]
    x_line = np.linspace(0, df['Credentials'].max() * 1.1, 100)

    # Proving linear scaling
    base_prove = df['Prove Time (ms)'].iloc[0]
    y_prove = (base_prove/base_creds) * x_line
    plt.plot(x_line, y_prove, '--', color='blue',
             label='Ideal Linear (Prove)', alpha=0.5)

    # Verification linear scaling
    base_verify = df['Verify Time (ms)'].iloc[0]
    y_verify = (base_verify/base_creds) * x_line
    plt.plot(x_line, y_verify, '--', color='red',
             label='Ideal Linear (Verify)', alpha=0.5)

    plt.xlabel('Number of Credentials')
    plt.ylabel('Execution Time (ms)')
    plt.title('PS Protocol Credential Scaling Analysis')
    plt.grid(True, alpha=0.3)
    plt.legend()

    # Save the visualization
    plt.savefig('ps_credential_scaling_analysis.png',
                dpi=300, bbox_inches='tight')
    plt.close()

    # Print detailed scaling analysis
    print("\nScaling Analysis:")
    print("-" * 100)
    print(f"{'Credentials':<12} | {'Prove Time':<20} | {'Verify Time':<20} | {'Prove Scaling':<15} | {'Verify Scaling':<15}")
    print("-" * 100)

    for i in range(1, len(df)):
        prev = df.iloc[i-1]
        curr = df.iloc[i]

        cred_ratio = curr['Credentials'] / prev['Credentials']
        prove_ratio = curr['Prove Time (ms)'] / prev['Prove Time (ms)']
        verify_ratio = curr['Verify Time (ms)'] / prev['Verify Time (ms)']

        prove_scaling = prove_ratio / cred_ratio
        verify_scaling = verify_ratio / cred_ratio

        print(f"{curr['Credentials']:<12} | {curr['Prove Time (ms)']:>8.2f} ms | {curr['Verify Time (ms)']:>8.2f} ms | {prove_scaling:>13.2f}x | {verify_scaling:>13.2f}x")


def main():
    print("Starting PS Protocol Benchmark Analysis")
    df = extract_benchmark_data()
    if df is not None:
        analyze_scaling(df)
        print("\nAnalysis complete! Check ps_credential_scaling_analysis.png for visualization")


if __name__ == "__main__":
    main()

Starting PS Protocol Benchmark Analysis
Analyzing benchmark data from: ../target/criterion/ps_equality_split

Found 5 proving and 5 verification configurations

Analyzing proving configuration: 30
Reading from: ../target/criterion/ps_equality_split/prove_credentials_30/new/estimates.json
✓ Successfully processed data for 30 credentials

Analyzing proving configuration: 2
Reading from: ../target/criterion/ps_equality_split/prove_credentials_2/new/estimates.json
✓ Successfully processed data for 2 credentials

Analyzing proving configuration: 5
Reading from: ../target/criterion/ps_equality_split/prove_credentials_5/new/estimates.json
✓ Successfully processed data for 5 credentials

Analyzing proving configuration: 20
Reading from: ../target/criterion/ps_equality_split/prove_credentials_20/new/estimates.json
✓ Successfully processed data for 20 credentials

Analyzing proving configuration: 10
Reading from: ../target/criterion/ps_equality_split/prove_credentials_10/new/estimates.json
✓ Suc

# PSUTT

In [4]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def extract_benchmark_data():
    """
    Extract benchmark data for both proving and verification times
    """
    results = []
    base_path = Path("../target/criterion/psutt_split")

    print(f"Analyzing benchmark data from: {base_path}")

    # Look for both prove and verify benchmarks
    prove_dirs = list(base_path.glob("prove_credentials_*"))
    verify_dirs = list(base_path.glob("verify_credentials_*"))

    print(
        f"\nFound {len(prove_dirs)} proving and {len(verify_dirs)} verification configurations")

    # Process proving benchmarks
    for bench_dir in prove_dirs:
        # Extract credential count from directory name
        creds = int(bench_dir.name.split('_')[2])

        estimates_file = bench_dir / "new" / "estimates.json"
        print(f"\nAnalyzing proving configuration: {creds}")
        print(f"Reading from: {estimates_file}")

        if estimates_file.exists():
            try:
                with open(estimates_file, 'r') as f:
                    data = json.load(f)

                # Extract timing metrics (converting nanoseconds to milliseconds)
                mean_time = data['mean']['point_estimate'] / 1_000_000
                std_dev = data['mean']['standard_error'] / 1_000_000

                # Find corresponding verification time
                verify_dir = next(d for d in verify_dirs if int(
                    d.name.split('_')[2]) == creds)
                verify_estimates = verify_dir / "new" / "estimates.json"

                with open(verify_estimates, 'r') as f:
                    verify_data = json.load(f)

                verify_mean = verify_data['mean']['point_estimate'] / 1_000_000
                verify_std = verify_data['mean']['standard_error'] / 1_000_000

                results.append({
                    'Credentials': creds,
                    'Prove Time (ms)': mean_time,
                    'Prove Std Dev': std_dev,
                    'Verify Time (ms)': verify_mean,
                    'Verify Std Dev': verify_std,
                    'Total Time (ms)': mean_time + verify_mean
                })
                print(f"✓ Successfully processed data for {creds} credentials")

            except Exception as e:
                print(f"× Error processing data: {e}")

    if results:
        # Create DataFrame and sort by credential count
        df = pd.DataFrame(results)
        df = df.sort_values('Credentials')

        # Save results
        df.to_csv('psutt_prover_verifier.csv', index=False)
        df.to_json('psutt_prover_verifier.json', orient='records', indent=2)

        print("\nScaling Analysis Summary:")
        print("-" * 70)
        print(df.to_string(index=False))

        return df
    else:
        print("\nNo data was collected!")
        return None


def analyze_scaling(df):
    """Analyze and visualize scaling behavior for both proving and verification"""
    plt.figure(figsize=(12, 8))

    # Plot proving time
    plt.scatter(df['Credentials'], df['Prove Time (ms)'],
                color='blue', label='Proving Time', s=100)
    if 'Prove Std Dev' in df.columns:
        plt.errorbar(df['Credentials'], df['Prove Time (ms)'],
                     yerr=df['Prove Std Dev'], fmt='none',
                     capsize=5, color='blue', alpha=0.5)

    # Plot verification time
    plt.scatter(df['Credentials'], df['Verify Time (ms)'],
                color='red', label='Verification Time', s=100)
    if 'Verify Std Dev' in df.columns:
        plt.errorbar(df['Credentials'], df['Verify Time (ms)'],
                     yerr=df['Verify Std Dev'], fmt='none',
                     capsize=5, color='red', alpha=0.5)

    # Plot total time
    plt.scatter(df['Credentials'], df['Total Time (ms)'],
                color='green', label='Total Time', s=100)

    # Compute and plot ideal linear scaling for both operations
    base_creds = df['Credentials'].iloc[0]
    x_line = np.linspace(0, df['Credentials'].max() * 1.1, 100)

    # Proving linear scaling
    base_prove = df['Prove Time (ms)'].iloc[0]
    y_prove = (base_prove/base_creds) * x_line
    plt.plot(x_line, y_prove, '--', color='blue',
             label='Ideal Linear (Prove)', alpha=0.5)

    # Verification linear scaling
    base_verify = df['Verify Time (ms)'].iloc[0]
    y_verify = (base_verify/base_creds) * x_line
    plt.plot(x_line, y_verify, '--', color='red',
             label='Ideal Linear (Verify)', alpha=0.5)

    plt.xlabel('Number of Credentials')
    plt.ylabel('Execution Time (ms)')
    plt.title('PSUtt Credential Scaling Analysis: Proving vs Verification')
    plt.grid(True, alpha=0.3)
    plt.legend()

    # Save the visualization
    plt.savefig('psutt_prover_verifier.png',
                dpi=300, bbox_inches='tight')
    plt.close()

    # Print scaling analysis
    print("\nScaling Analysis:")
    print("-" * 100)
    print(f"{'Credentials':<12} | {'Prove Time':<20} | {'Verify Time':<20} | {'Prove Scaling':<15} | {'Verify Scaling':<15}")
    print("-" * 100)

    for i in range(1, len(df)):
        prev = df.iloc[i-1]
        curr = df.iloc[i]

        cred_ratio = curr['Credentials'] / prev['Credentials']
        prove_ratio = curr['Prove Time (ms)'] / prev['Prove Time (ms)']
        verify_ratio = curr['Verify Time (ms)'] / prev['Verify Time (ms)']

        prove_scaling = prove_ratio / cred_ratio
        verify_scaling = verify_ratio / cred_ratio

        print(f"{curr['Credentials']:<12} | {curr['Prove Time (ms)']:>8.2f} ms | {curr['Verify Time (ms)']:>8.2f} ms | {prove_scaling:>13.2f}x | {verify_scaling:>13.2f}x")


def main():
    df = extract_benchmark_data()
    if df is not None:
        analyze_scaling(df)


if __name__ == "__main__":
    main()

Analyzing benchmark data from: ../target/criterion/psutt_split

Found 5 proving and 5 verification configurations

Analyzing proving configuration: 30
Reading from: ../target/criterion/psutt_split/prove_credentials_30/new/estimates.json
✓ Successfully processed data for 30 credentials

Analyzing proving configuration: 2
Reading from: ../target/criterion/psutt_split/prove_credentials_2/new/estimates.json
✓ Successfully processed data for 2 credentials

Analyzing proving configuration: 5
Reading from: ../target/criterion/psutt_split/prove_credentials_5/new/estimates.json
✓ Successfully processed data for 5 credentials

Analyzing proving configuration: 20
Reading from: ../target/criterion/psutt_split/prove_credentials_20/new/estimates.json
✓ Successfully processed data for 20 credentials

Analyzing proving configuration: 10
Reading from: ../target/criterion/psutt_split/prove_credentials_10/new/estimates.json
✓ Successfully processed data for 10 credentials

Scaling Analysis Summary:
-----

# PSUTT Improved

In [6]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def extract_benchmark_data():
    """
    Extract benchmark data for both proving and verification times
    """
    results = []
    base_path = Path("../target/criterion/psutt_improved_equality_split")

    print(f"Analyzing benchmark data from: {base_path}")

    # Look for both prove and verify benchmarks
    prove_dirs = list(base_path.glob("prove_credentials_*"))
    verify_dirs = list(base_path.glob("verify_credentials_*"))

    print(
        f"\nFound {len(prove_dirs)} proving and {len(verify_dirs)} verification configurations")

    # Process proving benchmarks
    for bench_dir in prove_dirs:
        # Extract credential count from directory name
        creds = int(bench_dir.name.split('_')[2])

        estimates_file = bench_dir / "new" / "estimates.json"
        print(f"\nAnalyzing proving configuration: {creds}")
        print(f"Reading from: {estimates_file}")

        if estimates_file.exists():
            try:
                with open(estimates_file, 'r') as f:
                    data = json.load(f)

                # Extract timing metrics (converting nanoseconds to milliseconds)
                mean_time = data['mean']['point_estimate'] / 1_000_000
                std_dev = data['mean']['standard_error'] / 1_000_000

                # Find corresponding verification time
                verify_dir = next(d for d in verify_dirs if int(
                    d.name.split('_')[2]) == creds)
                verify_estimates = verify_dir / "new" / "estimates.json"

                with open(verify_estimates, 'r') as f:
                    verify_data = json.load(f)

                verify_mean = verify_data['mean']['point_estimate'] / 1_000_000
                verify_std = verify_data['mean']['standard_error'] / 1_000_000

                results.append({
                    'Credentials': creds,
                    'Prove Time (ms)': mean_time,
                    'Prove Std Dev': std_dev,
                    'Verify Time (ms)': verify_mean,
                    'Verify Std Dev': verify_std,
                    'Total Time (ms)': mean_time + verify_mean
                })
                print(f"✓ Successfully processed data for {creds} credentials")

            except Exception as e:
                print(f"× Error processing data: {e}")

    if results:
        # Create DataFrame and sort by credential count
        df = pd.DataFrame(results)
        df = df.sort_values('Credentials')

        # Save results
        df.to_csv('psutt_improved_prover_verifier.csv', index=False)
        df.to_json('psutt_improved_prover_verifier.json',
                   orient='records', indent=2)

        print("\nPSUTT Improved Scaling Analysis Summary:")
        print("-" * 70)
        print(df.to_string(index=False))

        return df
    else:
        print("\nNo data was collected!")
        return None


def analyze_scaling(df):
    """Analyze and visualize scaling behavior for both proving and verification"""
    plt.figure(figsize=(12, 8))

    # Plot proving time
    plt.scatter(df['Credentials'], df['Prove Time (ms)'],
                color='blue', label='Proving Time', s=100)
    if 'Prove Std Dev' in df.columns:
        plt.errorbar(df['Credentials'], df['Prove Time (ms)'],
                     yerr=df['Prove Std Dev'], fmt='none',
                     capsize=5, color='blue', alpha=0.5)

    # Plot verification time
    plt.scatter(df['Credentials'], df['Verify Time (ms)'],
                color='red', label='Verification Time', s=100)
    if 'Verify Std Dev' in df.columns:
        plt.errorbar(df['Credentials'], df['Verify Time (ms)'],
                     yerr=df['Verify Std Dev'], fmt='none',
                     capsize=5, color='red', alpha=0.5)

    # Plot total time
    plt.scatter(df['Credentials'], df['Total Time (ms)'],
                color='green', label='Total Time', s=100)

    # Compute and plot ideal linear scaling for both operations
    base_creds = df['Credentials'].iloc[0]
    x_line = np.linspace(0, df['Credentials'].max() * 1.1, 100)

    # Proving linear scaling
    base_prove = df['Prove Time (ms)'].iloc[0]
    y_prove = (base_prove/base_creds) * x_line
    plt.plot(x_line, y_prove, '--', color='blue',
             label='Ideal Linear (Prove)', alpha=0.5)

    # Verification linear scaling
    base_verify = df['Verify Time (ms)'].iloc[0]
    y_verify = (base_verify/base_creds) * x_line
    plt.plot(x_line, y_verify, '--', color='red',
             label='Ideal Linear (Verify)', alpha=0.5)

    plt.xlabel('Number of Credentials')
    plt.ylabel('Execution Time (ms)')
    plt.title('PSUTT Improved Proving vs Verification')
    plt.grid(True, alpha=0.3)
    plt.legend()

    # Save the visualization
    plt.savefig('psutt_improved_prover_verifier.png',
                dpi=300, bbox_inches='tight')
    plt.close()

    # Print scaling analysis
    print("\nScaling Analysis:")
    print("-" * 100)
    print(f"{'Credentials':<12} | {'Prove Time':<20} | {'Verify Time':<20} | {'Prove Scaling':<15} | {'Verify Scaling':<15}")
    print("-" * 100)

    for i in range(1, len(df)):
        prev = df.iloc[i-1]
        curr = df.iloc[i]

        cred_ratio = curr['Credentials'] / prev['Credentials']
        prove_ratio = curr['Prove Time (ms)'] / prev['Prove Time (ms)']
        verify_ratio = curr['Verify Time (ms)'] / prev['Verify Time (ms)']

        prove_scaling = prove_ratio / cred_ratio
        verify_scaling = verify_ratio / cred_ratio

        print(f"{curr['Credentials']:<12} | {curr['Prove Time (ms)']:>8.2f} ms | {curr['Verify Time (ms)']:>8.2f} ms | {prove_scaling:>13.2f}x | {verify_scaling:>13.2f}x")


def main():
    df = extract_benchmark_data()
    if df is not None:
        analyze_scaling(df)


if __name__ == "__main__":
    main()

Analyzing benchmark data from: ../target/criterion/psutt_improved_equality_split

Found 5 proving and 5 verification configurations

Analyzing proving configuration: 30
Reading from: ../target/criterion/psutt_improved_equality_split/prove_credentials_30/new/estimates.json
✓ Successfully processed data for 30 credentials

Analyzing proving configuration: 2
Reading from: ../target/criterion/psutt_improved_equality_split/prove_credentials_2/new/estimates.json
✓ Successfully processed data for 2 credentials

Analyzing proving configuration: 5
Reading from: ../target/criterion/psutt_improved_equality_split/prove_credentials_5/new/estimates.json
✓ Successfully processed data for 5 credentials

Analyzing proving configuration: 20
Reading from: ../target/criterion/psutt_improved_equality_split/prove_credentials_20/new/estimates.json
✓ Successfully processed data for 20 credentials

Analyzing proving configuration: 10
Reading from: ../target/criterion/psutt_improved_equality_split/prove_credent

# Comparison PSUTT and PSUTT Improved

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def load_data(original_file, improved_file):
    """Load both datasets for comparison"""
    df_original = pd.read_csv(original_file)
    df_improved = pd.read_csv(improved_file)
    return df_original, df_improved

def create_comparison_plot(df_original, df_improved):
    """Create a detailed comparison plot with error bars"""
    plt.figure(figsize=(12, 8))
    
    # Set width of bars and positions of the bars
    bar_width = 0.15
    r1 = np.arange(len(df_original['Credentials']))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    r4 = [x + bar_width for x in r3]
    
    # Create bars
    plt.bar(r1, df_original['Prove Time (ms)'], bar_width, 
            label='Original Prove', color='blue', alpha=0.7,
            yerr=df_original['Prove Std Dev'], capsize=5)
    plt.bar(r2, df_improved['Prove Time (ms)'], bar_width,
            label='Improved Prove', color='green', alpha=0.7,
            yerr=df_improved['Prove Std Dev'], capsize=5)
    plt.bar(r3, df_original['Verify Time (ms)'], bar_width,
            label='Original Verify', color='red', alpha=0.7,
            yerr=df_original['Verify Std Dev'], capsize=5)
    plt.bar(r4, df_improved['Verify Time (ms)'], bar_width,
            label='Improved Verify', color='orange', alpha=0.7,
            yerr=df_improved['Verify Std Dev'], capsize=5)
    
    # Add labels and titles
    plt.xlabel('Number of Credentials')
    plt.ylabel('Time (ms)')
    plt.title('Performance Comparison: Original vs Improved Implementation')
    plt.xticks([r + bar_width*1.5 for r in range(len(df_original['Credentials']))],
               df_original['Credentials'])
    
    # Add legend
    plt.legend()
    
    # Add grid for better readability
    plt.grid(True, alpha=0.3)
    
    # Ensure layout fits well
    plt.tight_layout()
    
    # Save the plot
    plt.savefig('implementation_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

def analyze_improvements(df_original, df_improved):
    """Analyze and print improvement metrics"""
    improvements = pd.DataFrame({
        'Credentials': df_original['Credentials'],
        'Prove Improvement (%)': ((df_original['Prove Time (ms)'] - df_improved['Prove Time (ms)']) 
                                 / df_original['Prove Time (ms)'] * 100),
        'Verify Improvement (%)': ((df_original['Verify Time (ms)'] - df_improved['Verify Time (ms)'])
                                  / df_original['Verify Time (ms)'] * 100),
        'Total Improvement (%)': ((df_original['Total Time (ms)'] - df_improved['Total Time (ms)'])
                                 / df_original['Total Time (ms)'] * 100)
    })
    
    print("\nPerformance Improvement Analysis:")
    print("=" * 80)
    print(improvements.round(2).to_string(index=False))
    
    # Analyze scaling behavior
    print("\nScaling Analysis:")
    print("=" * 80)
    for i in range(1, len(df_original)):
        cred_ratio = df_original['Credentials'].iloc[i] / df_original['Credentials'].iloc[i-1]
        
        orig_prove_ratio = df_original['Prove Time (ms)'].iloc[i] / df_original['Prove Time (ms)'].iloc[i-1]
        orig_verify_ratio = df_original['Verify Time (ms)'].iloc[i] / df_original['Verify Time (ms)'].iloc[i-1]
        
        imp_prove_ratio = df_improved['Prove Time (ms)'].iloc[i] / df_improved['Prove Time (ms)'].iloc[i-1]
        imp_verify_ratio = df_improved['Verify Time (ms)'].iloc[i] / df_improved['Verify Time (ms)'].iloc[i-1]
        
        print(f"\nCredential increase {df_original['Credentials'].iloc[i-1]} → {df_original['Credentials'].iloc[i]}:")
        print(f"Original Prove scaling factor: {orig_prove_ratio/cred_ratio:.2f}x per credential ratio")
        print(f"Improved Prove scaling factor: {imp_prove_ratio/cred_ratio:.2f}x per credential ratio")
        print(f"Original Verify scaling factor: {orig_verify_ratio/cred_ratio:.2f}x per credential ratio")
        print(f"Improved Verify scaling factor: {imp_verify_ratio/cred_ratio:.2f}x per credential ratio")

def main():
    # Load both datasets
    df_original, df_improved = load_data('psutt_credential_scaling.csv', 
                                       'psutt_improved_prover_verifier.csv')
    
    # Create visualization
    create_comparison_plot(df_original, df_improved)
    
    # Analyze improvements
    analyze_improvements(df_original, df_improved)

if __name__ == "__main__":
    main()


Performance Improvement Analysis:
 Credentials  Prove Improvement (%)  Verify Improvement (%)  Total Improvement (%)
           2                   0.11                    7.35                   4.19
           5                   7.97                    2.16                   4.86
          10                   3.81                    0.87                   2.21
          20                   5.95                    2.25                   3.95
          30                   3.04                    2.63                   2.81

Scaling Analysis:

Credential increase 2 → 5:
Original Prove scaling factor: 1.06x per credential ratio
Improved Prove scaling factor: 0.98x per credential ratio
Original Verify scaling factor: 0.95x per credential ratio
Improved Verify scaling factor: 1.00x per credential ratio

Credential increase 5 → 10:
Original Prove scaling factor: 0.95x per credential ratio
Improved Prove scaling factor: 0.99x per credential ratio
Original Verify scaling factor: 0.99x per